In [1]:
# from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.formatting import Rule
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def sum_of_digits(i):
    sum_of_digits = 0
    i = str(i)
    for digit in i:
        sum_of_digits += int(digit)
    return sum_of_digits
  
def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    seed = "abc"
    val = hash(str(matr) + seed) % 10000000000
    if ws["BB3"].value == val:
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "          "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"

def check_exercise_1_1(wb, matr):
    try:
        ws = wb.worksheets[0]
        print(matr)
        points = 0
        # Check if the data in columnA has been "text in columned"
        if ws["B2"].value != None and ws["C2"].value != None:
            points += 1
        if ws["D2"].value != None and ws["D2"].value in ["m", "f", "d"]:
            points += 10
            
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_1")
        return "NOK"

def check_exercise_1_2(wb_val, matr):
    try:
        ws = wb_val.worksheets[0]
        points = 0
        # Check, if the salutation in column J is correct
        salutation_correct = True
        for row_ind in range(2, 200):
            if ";" in ws["A" + str(row_ind)].value:
                name = ws["A" + str(row_ind)].value.split(";")[0]
                sex = ws["A" + str(row_ind)].value.split(";")[-1]
            else:
                name = ws["A" + str(row_ind)].value
                sex = ws["D" + str(row_ind)].value
            if sex == "m":
                if ws["J" + str(row_ind)].value != "Mr. " + name:
                    salutation_correct = False
            elif sex == "f":
                if ws["J" + str(row_ind)].value != "Mrs. " + name:
                    salutation_correct = False
            elif sex == "d":
                if ws["J" + str(row_ind)].value != name:
                    salutation_correct = False

        if salutation_correct:
            return 4
        else:
            return 0
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"

def check_exercise_2_1(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        # Check if data is split into columns
        # B2 is 3-Letter-Country-Code
        if ws["B2"].value != None and not "B2" in ws.array_formulae and len(ws["B2"].value) == 3:
            points += 1
        # C50 is Continent
        if ws["C50"].value != None and ws["C50"].value in ["Africa", "Asia", "Europe", "North America", "South America", "Oceania"]:
            points += 10
        # D100 is Year
        if ws["D100"].value != None and ws["D100"].value in [1990, 2000, 2010, 2020, "1990", "2000", "2010", "2020"]:
            points += 100
        # E150 is float or integer
        if ws["E150"].value != None:
            x = ws["E150"].value
            if isinstance(x, (float, int)):
                points += 1000

        # Check if data is sorted with respect to column C (continents)
        l_continents = []
        for row_ind in range(2, 100):
            if ws["C" + str(row_ind)].value != None and ws["D" + str(row_ind)].value != None and ws["D" + str(row_ind)].value in [2020, "2020"]:
                l_continents.append(ws["C" + str(row_ind)].value)
        if len(l_continents) > 100 and l_continents == sorted(l_continents):
            points += 10000

        # If Auto-Filter are used check the status of the filter
        if ws.auto_filter.ref != None:
            for i in range(len(ws.auto_filter.filterColumn)):
                if len(ws.auto_filter.filterColumn[i].filters.filter) > 0:
                    if ws.auto_filter.filterColumn[i].filters.filter[0] in [2020, "2020", "Asia", "Europe"]:
                        points += 100000
        
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_1")
        return "NOK"

def check_exercise_2_2(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        bubble_chart = ""
        if len(ws._charts) >= 1 and "bubble_chart" in str(type(ws._charts[0])):
            bubble_chart = ws._charts[0]
            if bubble_chart.series != None:
                if bubble_chart.series[0].xVal != None and bubble_chart.series[0].xVal.numRef != None:
                    xval = bubble_chart.series[0].xVal.numRef.f
                    xval = xval.split("!")[1].replace("$", "")
                    if ":" in xval:
                        cell1, cell2 = xval.split(":")
                        if cell1[0] == "E" and cell2[0] == "E":
                            points += 1
                if bubble_chart.series[0].yVal != None and bubble_chart.series[0].yVal.numRef != None:
                    yval = bubble_chart.series[0].yVal.numRef.f
                    yval = yval.split("!")[1].replace("$", "")
                    if ":" in yval:
                        cell1, cell2 = yval.split(":")
                        if cell1[0] == "G" and cell2[0] == "G":
                            points += 10
                if bubble_chart.series[0].zVal != None and bubble_chart.series[0].zVal.numRef != None:
                    zval = bubble_chart.series[0].zVal.numRef.f
                    zval = zval.split("!")[1].replace("$", "")
                    if ":" in zval:
                        cell1, cell2 = zval.split(":")
                        if cell1[0] == "F" and cell2[0] == "F":
                            points += 100
                if bubble_chart.series[0].title != None and bubble_chart.series[0].title.value != None:
                    legend = bubble_chart.series[0].title.value
                    if legend in ["Africa", "Asia", "Europe", "North America", "South America", "Oceania"]:
                        points += 1000
                if bubble_chart.series[0].title != None and bubble_chart.series[0].title.strRef != None:
                    legend = bubble_chart.series[0].title.strRef.f
                    if "!" in legend:
                        legend = legend.split("!")[1].replace("$", "")
                        if legend[0] == "C":
                            points += 1000
                    elif legend in ["Africa", "Asia", "Europe", "North America", "South America", "Oceania"]:
                        points += 1000
                
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_2")
        return "NOK"


def check_exercise_3_1(wb_val, matr):
    try:
        ws = wb_val.worksheets[2]
        points = 0
        # Check if calculations are done correctly
        for row_ind in [23, 1000, 10172, 19926]:
            x = ws["A" + str(row_ind)].value
            y = ws["B" + str(row_ind)].value
            result = x**2 + y**2 - 1 - abs(x) * y
            if ws["C" + str(row_ind)].value != None:
                if round(ws["C" + str(row_ind)].value, 5) == round(result, 5):
                    points += 1
            if ws["D" + str(row_ind)].value != None and ws["D" + str(row_ind)].value != "#VALUE!":
                if -0.01 < result < 0.01:
                    final_result = 0
                else:
                    final_result = 1
                if final_result == int(ws["D" + str(row_ind)].value):
                    points += 10

        return sum_of_digits(points) // 2
    except:
        print("FAIL:", matr, "check_exercise_3_1")
        return "NOK"

def check_exercise_3_2(wb, matr):
    try:
        ws = wb.worksheets[2]
        points = 0
        if ws.auto_filter.ref != None:
            for i in range(len(ws.auto_filter.filterColumn)):
                if ws.auto_filter.filterColumn[i].filters != None:
                    if ws.auto_filter.filterColumn[i].filters.filter[0] in [0, "0"]:
                        points += 2

        return sum_of_digits(points) 
    except:
        print("FAIL:", matr, "check_exercise_3_2")
        return "NOK"

def check_exercise_3_3(wb, matr):
    try:
        ws = wb.worksheets[2]
        points = 0
        if len(ws._charts) >= 1 and "scatter_chart" in str(type(ws._charts[0])):
            scatter_chart = ws._charts[0]
            if len(scatter_chart.series) > 0:
                if scatter_chart.series[-1].xVal != None and scatter_chart.series[-1].xVal.numRef:
                    xaxis = scatter_chart.series[-1].xVal.numRef.f
                    xaxis = xaxis.split("!")[1].replace("$", "")
                    cell1, cell2 = xaxis.split(":")
                    if cell1[0] == "A" and cell2[0] == "A":
                        points += 1
                # cm in y-Achse
                if scatter_chart.series[-1].yVal != None and scatter_chart.series[-1].yVal.numRef != None:
                    yaxis = scatter_chart.series[-1].yVal.numRef.f
                    yaxis = yaxis.split("!")[1].replace("$", "")
                    cell1, cell2 = yaxis.split(":")
                    if cell1[0] == "B" and cell2[0] == "B":
                        points += 10

        return sum_of_digits(points) 
    except:
        print("FAIL:", matr, "check_exercise_3_3")
        return "NOK"


def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb_val, matr))
    points.append(check_exercise_2_1(wb, matr))
    points.append(check_exercise_2_2(wb, matr))
    points.append(check_exercise_3_1(wb_val, matr))
    points.append(check_exercise_3_2(wb, matr))
    points.append(check_exercise_3_3(wb, matr))
    
    return points

l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
    header = ["MatrNr", "Fingerprint", "Array", "1 Text in Column", "1 Salutation", "2 Text to Column, Sort, Filter", "2 Chart", "3 Calculation", "3 Filter", "3 Chart"] 
common.result_into_file(l_points, header)


3668942
3733682
3714810
3652468
3718566
3735433
3728866
3710474
3650295
3725401
3720435
3552737
3653692
3539743
3715086
3661857
3688878
3738393
3740473
3713701
3709333
3730508
3723816
3586255
3648470
3725810
3609318
3728048
3539194
3230373
3676446
3740501
3699472
3659595
3667338
3737721
3664564


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/reader/drawings.py:46: UserWarning: Unable to read chart rId2 from xl/drawings/drawing1.xml expected <class 'float'>
  warn(f"Unable to read chart {rel.id} from {path} {e}")


3735532
3710135
3735347
3730496
3652541
3672745
3720886
3733484
3713602
3734788
3714302
3726242
3723647
3731873
3720381
3651812
3717725
3659131
3736919
3722161
FAIL: 3722161 check_exercise_3_2
FAIL: 3722161 check_exercise_3_3
3724730
3665433
3591592
[3668942, 'NOK', '          ', 2, 4, 4, 3, 1, 2, 0]
[3733682, 'NOK', '          ', 0, 0, 5, 3, 4, 2, 2]
[3714810, 'NOK', '          ', 2, 4, 6, 3, 2, 2, 2]
[3652468, 'NOK', '          ', 0, 4, 6, 4, 3, 0, 0]
[3718566, 'NOK', '          ', 2, 4, 5, 1, 4, 0, 0]
[3735433, 'NOK', '          ', 2, 0, 0, 0, 0, 0, 0]
[3728866, 'NOK', '          ', 0, 4, 5, 2, 4, 0, 2]
[3710474, 'NOK', '          ', 2, 0, 4, 0, 4, 0, 0]
[3650295, 'NOK', '          ', 0, 4, 5, 2, 0, 0, 0]
[3725401, 'NOK', '          ', 2, 4, 6, 4, 4, 2, 1]
[3720435, 'NOK', '          ', 0, 4, 5, 4, 4, 2, 2]
[3552737, 'NOK', '          ', 2, 4, 5, 3, 4, 2, 2]
[3653692, 'NOK', '          ', 2, 4, 5, 3, 4, 2, 2]
[3539743, 'NOK', '          ', 1, 4, 5, 0, 4, 2, 2]
[3715086, 'NOK', '    

# os.chdir(".")
print(os.getcwd())